# Imports

In [1]:
import numpy as np 
import pandas as pd 
import os
import shutil
import json
import glob

!pip install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 4.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 77.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 67.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 33.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664

# Environment Setup and Directory Creation

## clear previous folders

In [2]:
import shutil, os

# Paths used by nnUNet
paths_to_clear = [
    "/kaggle/working/nnUNet_raw_data_base",
    "/kaggle/working/nnUNet_preprocessed",
    "/kaggle/working/nnUNet_results"
]

for p in paths_to_clear:
    if os.path.exists(p):
        print(f"🗑️ Removing {p} ...")
        shutil.rmtree(p)
print("✅ All previous nnUNet folders cleared.")

✅ All previous nnUNet folders cleared.


In [3]:
os.environ['nnUNet_raw_data_base'] = '/kaggle/working/nnUNet_raw_data_base'

# Create necessary directories
!mkdir -p $nnUNet_raw_data_base/nnUNet_raw/Dataset002_BonnFCD_FLAIR

print("nnUNet environment setup complete.")

nnUNet environment setup complete.


In [4]:
UPLOADED_DATASET_NAME = 'participants-data' 
CSV_DATASET_ROOT = f'/kaggle/input/{UPLOADED_DATASET_NAME}'

print(f"Listing files in the root of your uploaded metadata dataset ({CSV_DATASET_ROOT}):")
if os.path.exists(CSV_DATASET_ROOT):
    # This will print the actual file name (e.g., ['participants.csv'] or ['participants.xlsx'])
    print(os.listdir(CSV_DATASET_ROOT))
else:
    print(f"Error: The dataset root {CSV_DATASET_ROOT} was not found. Please verify the uploaded dataset name.")

Listing files in the root of your uploaded metadata dataset (/kaggle/input/participants-data):
['participants.tsv', 'participants.xlsx']


# Data Conversion and dataset.json Generation

In [5]:
# --- 🎯 FINAL PATHS AND NAMES 🎯 ---
BASE_DIR = '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed' 
UPLOADED_DATASET_NAME = 'participants-data' 
EXCEL_FILE_NAME = 'participants.xlsx' 
EXCEL_PATH = os.path.join('/kaggle/input', UPLOADED_DATASET_NAME, EXCEL_FILE_NAME)
# ---------------------------------------------

TASK_ID = 2
TASK_NAME = 'Dataset002_BonnFCD_FLAIR' 
NNUNET_RAW_DATA_DIR = os.path.join(os.environ['nnUNet_raw_data_base'], 'nnUNet_raw', TASK_NAME) 

# Create target directories
# Create target directories
IMAGES_TR_DIR = os.path.join(NNUNET_RAW_DATA_DIR, 'imagesTr')
LABELS_TR_DIR = os.path.join(NNUNET_RAW_DATA_DIR, 'labelsTr')
IMAGES_TS_DIR = os.path.join(NNUNET_RAW_DATA_DIR, 'imagesTs')
LABELS_TS_DIR = os.path.join(NNUNET_RAW_DATA_DIR, 'labelsTs')  # optional test labels

# Create them if missing
for d in [IMAGES_TR_DIR, LABELS_TR_DIR, IMAGES_TS_DIR, LABELS_TS_DIR]:
    os.makedirs(d, exist_ok=True)

# --- 1. Load and Filter the Excel Data ---
print(f"Attempting to read Excel file from: {EXCEL_PATH}")

try:
    participants_df = pd.read_excel(EXCEL_PATH, sheet_name='participants')
except ValueError:
    xls = pd.ExcelFile(EXCEL_PATH)
    print(f"Available sheets: {xls.sheet_names}")
    participants_df = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
except Exception as e:
    print(f"FATAL ERROR: Could not read Excel file. Please verify the EXCEL_PATH: {EXCEL_PATH}. Error: {e}")
    raise

# Ensure required columns exist
required_columns = {'participant_id', 'group', 'split'}
missing_columns = required_columns - set(participants_df.columns)
if missing_columns:
    raise Exception(f"Missing required columns in Excel file: {missing_columns}")

# Filter FCD subjects for training and testing
train_fcd_subjects = participants_df[
    (participants_df['group'].str.lower() == 'fcd') & 
    (participants_df['split'].str.lower() == 'train')
]['participant_id'].tolist()

test_fcd_subjects = participants_df[
    (participants_df['group'].str.lower() == 'fcd') & 
    (participants_df['split'].str.lower() == 'test')
]['participant_id'].tolist()

print(f"Total FCD subjects for TRAINING: {len(train_fcd_subjects)}")
print(f"Total FCD subjects for TESTING: {len(test_fcd_subjects)}")

training_files = []
test_files = []
skipped_subjects = []

# --- Helper Function to Process Subjects ---
def process_subject(subject_id, target_images_dir, is_training=True):
    subject_path_with_anat = os.path.join(BASE_DIR, subject_id, 'anat')
    
    if not os.path.exists(subject_path_with_anat):
        return False, f"'anat' folder missing"

    # Look for relevant MRI and label files
    # Look for relevant FLAIR and ROI files only
    flair_search_pattern = os.path.join(subject_path_with_anat, f'{subject_id}*_FLAIR.nii')
    label_search_pattern = os.path.join(subject_path_with_anat, f'{subject_id}*_FLAIR_roi.nii')
    
    flair_files = glob.glob(flair_search_pattern)
    label_files = glob.glob(label_search_pattern)
    
    # Validation checks
    if len(flair_files) != 1:
        return False, f"Missing or ambiguous FLAIR (found {len(flair_files)})."
    
    if is_training and len(label_files) != 1:
        return False, f"Training subject missing label file (found {len(label_files)})."

    # Prepare target filenames
    flair_target_name = f'{subject_id}_0000.nii'  # FLAIR only
    label_target_name = f'{subject_id}.nii'
    
    try:
        # Copy MRI modalities
        shutil.copy(flair_files[0], os.path.join(target_images_dir, flair_target_name))

        if is_training:
            shutil.copy(label_files[0], os.path.join(LABELS_TR_DIR, label_target_name))
            return True, {"image": f"./imagesTr/{subject_id}", "label": f"./labelsTr/{subject_id}.nii"}
        else:
            return True, {"image": f"./imagesTs/{subject_id}"}
            
    except Exception as e:
        return False, f"Copy error: {e}"

# --- 2. Process Training and Test Subjects ---
print("\n--- Processing Training Subjects ---")
for subject_id in train_fcd_subjects:
    success, result = process_subject(subject_id, IMAGES_TR_DIR, is_training=True)
    if success:
        training_files.append(result)
    else:
        skipped_subjects.append((subject_id, f"TRAIN - {result}"))

print("\n--- Processing Test Subjects ---")
for subject_id in test_fcd_subjects:
    success, result = process_subject(subject_id, IMAGES_TS_DIR, is_training=False)
    if success:
        test_files.append(result)

    else:
        skipped_subjects.append((subject_id, f"TEST - {result}"))

# --- 3. Generate dataset.json ---
dataset_json = {
    "name": "FCD Lesion Segmentation",
    "description": "Focal Cortical Dysplasia Lesion Segmentation Dataset (Pre-defined Splits)",
    "reference": "your/publication/link/here",
    "licence": "CC-BY-4.0",
    "release": "1.0",
    "channel_names": {
    "0": "FLAIR"
    },
    "labels": {
        "background": 0,
        "lesion": 1
    },
    "numTraining": len(training_files),
    "file_ending": ".nii",
    "training": training_files,
    "test": test_files
}

# Save JSON
with open(os.path.join(NNUNET_RAW_DATA_DIR, 'dataset.json'), 'w') as f:
    json.dump(dataset_json, f, indent=4)

print(f"\n✅ Conversion complete!")
print(f"  Training subjects: {len(training_files)}")
print(f"  Test subjects: {len(test_files)}")

if skipped_subjects:
    print("\n--- ⚠️ Skipped Subjects Summary ---")
    for subj, reason in skipped_subjects:
        print(f"  {subj}: {reason}")

Attempting to read Excel file from: /kaggle/input/participants-data/participants.xlsx
Available sheets: ['Sheet1']
Total FCD subjects for TRAINING: 57
Total FCD subjects for TESTING: 28

--- Processing Training Subjects ---

--- Processing Test Subjects ---

✅ Conversion complete!
  Training subjects: 57
  Test subjects: 28


In [6]:
# --- Copy test labels if available (optional evaluation) ---
LABELS_TS_DIR = os.path.join(NNUNET_RAW_DATA_DIR, 'labelsTs')
os.makedirs(LABELS_TS_DIR, exist_ok=True)

for subject_id in test_fcd_subjects:
    anat_folder = os.path.join(BASE_DIR, subject_id, 'anat')
    label_search_pattern = os.path.join(anat_folder, f'{subject_id}*_FLAIR_roi.nii')
    label_files = glob.glob(label_search_pattern)
    
    if label_files:
        shutil.copy(label_files[0], os.path.join(LABELS_TS_DIR, f'{subject_id}.nii'))
        print(f"✅ Copied label for test subject {subject_id}")


✅ Copied label for test subject sub-00043
✅ Copied label for test subject sub-00066
✅ Copied label for test subject sub-00048
✅ Copied label for test subject sub-00064
✅ Copied label for test subject sub-00061
✅ Copied label for test subject sub-00092
✅ Copied label for test subject sub-00099
✅ Copied label for test subject sub-00032
✅ Copied label for test subject sub-00121
✅ Copied label for test subject sub-00136
✅ Copied label for test subject sub-00125
✅ Copied label for test subject sub-00134
✅ Copied label for test subject sub-00006
✅ Copied label for test subject sub-00144
✅ Copied label for test subject sub-00020
✅ Copied label for test subject sub-00142
✅ Copied label for test subject sub-00107
✅ Copied label for test subject sub-00074
✅ Copied label for test subject sub-00128
✅ Copied label for test subject sub-00108
✅ Copied label for test subject sub-00009
✅ Copied label for test subject sub-00004
✅ Copied label for test subject sub-00095
✅ Copied label for test subject su

# Data Preprocessing

## 1. Set nnU-Net environment paths

In [7]:
import os, subprocess

# Set environment variables for nnU-Net
os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw_data_base/nnUNet_raw"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"

print("✅ nnU-Net environment variables set")
print("RAW:", os.environ["nnUNet_raw"])

✅ nnU-Net environment variables set
RAW: /kaggle/working/nnUNet_raw_data_base/nnUNet_raw


## 2. Verify dataset integrity

In [8]:
print("\n🔍 Verifying dataset structure for Dataset002_BonnFCD_FLAIR...")
verify_command = [
    "nnUNetv2_plan_and_preprocess",
    "-d", "2",  # Dataset002_
    "--verify_dataset_integrity"
]

subprocess.run(verify_command, check=False)


🔍 Verifying dataset structure for Dataset002_BonnFCD_FLAIR...
Fingerprint extraction...
Dataset002_BonnFCD_FLAIR
Using <class 'nnunetv2.imageio.nibabel_reader_writer.NibabelIO'> as reader/writer
Affine image: [[ 9.94139194e-01  5.02954759e-02  9.56572741e-02 -1.05817863e+02]
 [-5.00640646e-02  9.98734355e-01 -4.81724599e-03 -9.16165466e+01]
 [-9.57781449e-02  2.11276010e-08  9.95402634e-01 -9.58630219e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]] 
Affine seg: [[ 9.94142830e-01  5.02954759e-02  9.56572741e-02 -1.05818451e+02]
 [-5.00642471e-02  9.98734355e-01 -4.81724599e-03 -9.16165161e+01]
 [-9.57784951e-02  0.00000000e+00  9.95402634e-01 -9.58629761e+01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
Image files: ['/kaggle/working/nnUNet_raw_data_base/nnUNet_raw/Dataset002_BonnFCD_FLAIR/imagesTr/sub-00116_0000.nii']. 
Seg file: /kaggle/working/nnUNet_raw_data_base/nnUNet_raw/Dataset002_BonnFCD_FLAIR/labelsTr/sub-00116.nii.
This can be 

100%|██████████| 57/57 [00:27<00:00,  2.04it/s]


Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [1.03 1.03 1.03]. 
Current patch size: (160, 160, 96). 
Current median shape: [247.57281553 247.57281553 155.33980583]
Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [255. 255. 160.], 3d_lowres: [248, 248, 155]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 80, 'patch_size': (256, 160), 'median_image_size_in_voxels': array([255., 160.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['ZScoreNormalization'], 'use_mask_for_norm': [False], 'resampling_fn_data': 'resample_data_or_seg_to_shape', 'resampling

100%|██████████| 57/57 [04:35<00:00,  4.83s/it]


Configuration: 3d_fullres...


100%|██████████| 57/57 [03:49<00:00,  4.03s/it]


Configuration: 3d_lowres...
INFO: Configuration 3d_lowres not found in plans file nnUNetPlans.json of dataset Dataset002_BonnFCD_FLAIR. Skipping.


CompletedProcess(args=['nnUNetv2_plan_and_preprocess', '-d', '2', '--verify_dataset_integrity'], returncode=0)

## 3. Run preprocessing

In [9]:
print("\n⚙️ Starting preprocessing (this may take several minutes)...")

preprocess_command = [
    "nnUNetv2_plan_and_preprocess",
    "-d", "2"
]

subprocess.run(preprocess_command, check=False)

print("\n✅ Preprocessing complete!")
print("Preprocessed data stored in: /kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR/")



⚙️ Starting preprocessing (this may take several minutes)...
Fingerprint extraction...
Dataset002_BonnFCD_FLAIR
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [1.03 1.03 1.03]. 
Current patch size: (160, 160, 96). 
Current median shape: [247.57281553 247.57281553 155.33980583]
Dropping 3d_lowres config because the image size difference to 3d_fullres is too small. 3d_fullres: [255. 255. 160.], 3d_lowres: [248, 248, 155]
2D U-Net configuration:
{'data_identifier': 'nnUNetPlans_2d', 'preprocessor_name': 'DefaultPreprocessor', 'batch_size': 80, 'patch_size': (256, 160), 'median_image_size_in_voxels': array([255., 160.]), 'spacing': array([1., 1.]), 'normalization_schemes': ['ZScore

100%|██████████| 57/57 [04:48<00:00,  5.06s/it]


Configuration: 3d_fullres...


100%|██████████| 57/57 [03:52<00:00,  4.09s/it]


Configuration: 3d_lowres...
INFO: Configuration 3d_lowres not found in plans file nnUNetPlans.json of dataset Dataset002_BonnFCD_FLAIR. Skipping.

✅ Preprocessing complete!
Preprocessed data stored in: /kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR/


In [10]:
# 4. (Optional) Verify output paths
import os

pre_dir = "/kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD_FLAIR"
if os.path.exists(pre_dir):
    print("✅ Folder exists! Contents:")
    print(os.listdir(pre_dir))
else:
    print("❌ Preprocessed folder not found!")


✅ Folder exists! Contents:
['gt_segmentations', 'nnUNetPlans_3d_fullres', 'nnUNetPlans.json', 'dataset_fingerprint.json', 'dataset.json', 'nnUNetPlans_2d']
